# 주간/주말 박스오피스 데이터


최근 50주간 데이터 중 주간 박스오피스 TOP10데이터 수집

* 조건
    1. 주간(월 ~ 일) `weekGb : “0” : 주간 (월~일)`
    2. 조회기간 총 50주, 기준일 2019년 7월 13일 `targetDt`
    3. 다양성 영화/상업 영화 모두 포함(default)
    4. 한국/외국 영화 모두 포함(default)
    5. 모든 상영지역 포함(default)


* 결과
    * `영화 대표코드 : movieCd`, `영화명 : movieNm`, `해당일 누적관객수 : audiAcc`
    * 해당일 누적 관객수는 중복시 최신 정보로 반영 **(이미 있는 데이터라면 그냥 넘어가기)**
    * 해당 결과 boxoffice.csv에 저장
    
    
* [소스코드](./01.py)


In [1]:
# 20190713 주간 박스오피스 조회

import requests
import pprint
import datetime

key = '2c6010411226af12f598c9e149bfeca8'
weekGb = '0' # 주간
targetDt = '20190713'
api_url = f'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchWeeklyBoxOfficeList.json?key={key}&targetDt={targetDt}'
response = requests.get(api_url).json()
pprint.pprint(response)

{'boxOfficeResult': {'boxofficeType': '주말 박스오피스',
                     'showRange': '20190712~20190714',
                     'weeklyBoxOfficeList': [{'audiAcc': '6685136',
                                              'audiChange': '-54.4',
                                              'audiCnt': '1302522',
                                              'audiInten': '-1555300',
                                              'movieCd': '20196309',
                                              'movieNm': '스파이더맨: 파 프롬 홈',
                                              'openDt': '2019-07-02',
                                              'rank': '1',
                                              'rankInten': '0',
                                              'rankOldAndNew': 'OLD',
                                              'rnum': '1',
                                              'salesAcc': '57709310340',
                                              'salesAmt': '11554695230',
        

In [2]:
# 20190713부터 50주 동안의 주간 박스오피스 조회
import requests
import csv
import datetime

key = '2c6010411226af12f598c9e149bfeca8'
weekGb = '0' # 주간
movies = {} # 50 주간 영화 저장할 딕셔너리

# 2019년 7월 13일을 기준으로 week주 전 날짜(dt)를 반환해주는 함수
def target_date(week):
    dt = datetime.datetime(2019, 7, 13) - datetime.timedelta(weeks=week)
    return dt.strftime('%Y%m%d')

In [3]:
# 1주 전 날짜
target_date(1)

'20190706'

In [4]:
# 주간 리스트 받아오기 for문 시작
for week in range(49, -1, -1): # 49주전부터 0주전(현재 2019.07.13)까지 50주 동안
    # targetDT 받아오는 함수 호출
    targetDt = target_date(week)
    api_url = f'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchWeeklyBoxOfficeList.json?key={key}&targetDt={targetDt}&weekGb={weekGb}'
    response = requests.get(api_url).json()
    
    # 반복문 실행하여 영화 대표코드, 영화 이름, 누적관객수를 변수에 저장함
    for movie_info in response.get('boxOfficeResult').get('weeklyBoxOfficeList'):
        movieCd = movie_info.get('movieCd')
        movieNm = movie_info.get('movieNm')
        audiAcc = movie_info.get('audiAcc')
        # 키(영화 대표코드), 값(영화 정보 - 영화 대표코드, 영화명, 누적관객수) 형태의 딕셔너리를 리스트 안에 삽입
        movies[movieCd] = {'movieCd': movieCd, 'movieNm': movieNm, 'audiAcc': audiAcc}

In [5]:
import pprint
pprint.pprint(movies)

{'19880001': {'audiAcc': '142372', 'movieCd': '19880001', 'movieNm': '이웃집 토토로'},
 '19990220': {'audiAcc': '49958', 'movieCd': '19990220', 'movieNm': '노팅 힐'},
 '20010291': {'audiAcc': '259733',
              'movieCd': '20010291',
              'movieNm': '해리포터와 마법사의 돌'},
 '20020222': {'audiAcc': '79301',
              'movieCd': '20020222',
              'movieNm': '해리포터와 비밀의 방'},
 '20060347': {'audiAcc': '482408',
              'movieCd': '20060347',
              'movieNm': '판의 미로 - 오필리아와 세 개의 열쇠'},
 '20139221': {'audiAcc': '3262924', 'movieCd': '20139221', 'movieNm': '그래비티'},
 '20153446': {'audiAcc': '2812226', 'movieCd': '20153446', 'movieNm': '너의 결혼식'},
 '20156554': {'audiAcc': '702096', 'movieCd': '20156554', 'movieNm': '물괴'},
 '20162003': {'audiAcc': '83376',
              'movieCd': '20162003',
              'movieNm': '명탐정 코난:전율의 악보'},
 '20166301': {'audiAcc': '26086', 'movieCd': '20166301', 'movieNm': '썬키스 패밀리'},
 '20167903': {'audiAcc': '439910', 'movieCd': '20167903', 'movi

In [ ]:
# boxoffice.csv 결과 파일 저장
with open('boxoffice.csv', 'w', encoding='utf-8') as f:
    fieldnames = ['movieCd', 'movieNm', 'audiAcc'] # 헤더와 딕셔너리 키 값을 맞춰줘야함.
    csv_writer = csv.DictWriter(f, fieldnames=fieldnames)
    csv_writer.writeheader()
    for movie in movies.values():
        csv_writer.writerow(movie)

In [7]:
# 결과 파일 확인
with open('boxoffice.csv', 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        print(row)

OrderedDict([('movieCd', '20186202'), ('movieNm', '신과함께-인과 연'), ('audiAcc', '12264813')])
OrderedDict([('movieCd', '20181181'), ('movieNm', '미션 임파서블: 폴아웃'), ('audiAcc', '6567099')])
OrderedDict([('movieCd', '20183361'), ('movieNm', '인크레더블 2'), ('audiAcc', '2998015')])
OrderedDict([('movieCd', '20186501'), ('movieNm', '극장판 헬로카봇 : 백악기 시대'), ('audiAcc', '854230')])
OrderedDict([('movieCd', '20185242'), ('movieNm', '신비아파트: 금빛 도깨비와 비밀의 동굴'), ('audiAcc', '667879')])
OrderedDict([('movieCd', '20170942'), ('movieNm', '인랑'), ('audiAcc', '891308')])
OrderedDict([('movieCd', '20181942'), ('movieNm', '맘마미아!2'), ('audiAcc', '2280120')])
OrderedDict([('movieCd', '20186121'), ('movieNm', '어느 가족'), ('audiAcc', '123887')])
OrderedDict([('movieCd', '20180522'), ('movieNm', '앤트맨과 와스프'), ('audiAcc', '5444801')])
OrderedDict([('movieCd', '20185341'), ('movieNm', '마녀'), ('audiAcc', '3188405')])
OrderedDict([('movieCd', '20179101'), ('movieNm', '공작'), ('audiAcc', '4968325')])
OrderedDict([('movieCd', '201848

# 영화 상세정보


01에서 수집한 영화 대표코드 활용해 상세 정보 수집.
향후 영화 정보로 활용될 것.

* 결과
    * 저장할 내용
        1.  `영화 대표코드` : movieCd
        2. `영화명(국문)` : movieNm
        3. `영화명(영문)` : movieNmEn	
        4. `영화명(원문)` : movieNmOg
        5. `관람등급` : watchGradeNm
        6. `개봉연도` : openDt
        7. `상영시간` : showTm
        8. `장르` : genreNm
        9. `감독명` : peopleNm
        

    * 해당 결과 movie.csv에 저장
    

* [소스코드](./02.py)

In [8]:
# 영화 상세정보

import requests
# import pprint
import csv

movie_infos = [] # 영화 상세정보 딕셔너리가 들어갈 리스트
key = '2c6010411226af12f598c9e149bfeca8'
api_url = f'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key={key}&movieCd='

In [9]:
# csv 열기
with open('boxoffice.csv', 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        temp = {}
        # url로 요청하고 값 받아오기
        response = requests.get(api_url + row.get('movieCd')).json()
        response = response.get('movieInfoResult').get('movieInfo')
        watchGradeNm = response.get('audits')[0].get('watchGradeNm') if response.get('audits') else None
        genreNm = response.get('genres')[0].get('genreNm') if response.get('genres') else None
        peopleNm = response.get('directors')[0].get('peopleNm') if response.get('directors') else None
        # 한 영화의 상세 정보 딕셔너리
        temp = {
            'movieCd': response.get('movieCd'),
            'movieNm': response.get('movieNm'),
            'movieNmEn': response.get('movieNmEn'),
            'movieNmOg': response.get('movieNmOg'),
            'watchGradeNm': watchGradeNm,
            'openDt': response.get('openDt'),
            'showTm': response.get('showTm'),
            'genreNm': genreNm,
            'peopleNm': peopleNm
        }
        movie_infos.append(temp)

In [12]:
import pprint
pprint.pprint(response)

{'actors': [{'cast': '',
             'castEn': '',
             'peopleNm': '플로렌스 퓨',
             'peopleNmEn': 'Florence Pugh'},
            {'cast': '',
             'castEn': '',
             'peopleNm': '잭 레이너',
             'peopleNmEn': 'Jack Reynor'},
            {'cast': '',
             'castEn': '',
             'peopleNm': '윌 폴터',
             'peopleNmEn': 'Will Poulter'},
            {'cast': '',
             'castEn': '',
             'peopleNm': '윌리엄 잭슨 하퍼',
             'peopleNmEn': 'William Jackson Harper'}],
 'audits': [{'auditNo': '2019-MF01288', 'watchGradeNm': '청소년관람불가'}],
 'companys': [{'companyCd': '20122837',
               'companyNm': '(주)팝엔터테인먼트',
               'companyNmEn': 'Pop Entertainment',
               'companyPartNm': '배급사'},
              {'companyCd': '20061718',
               'companyNm': '찬란',
               'companyNmEn': 'Challan Film',
               'companyPartNm': '수입사'},
              {'companyCd': '20122837',
               'company

01.py와 다르게 반복문을 이용하지 않는다.
response에는 한 영화의 상세정보를 담고 있다.
필요한 영화 대표코드, 영화명, 영화명(영문), 영화명(원문), 관람등급, 개봉연도, 상영시간, 장르, 감독명만 뽑아 딕셔너리(temp)로 저장한 후 movie_infos 리스트에 삽입한다.

In [13]:
# 영화 상세정보 movie.csv에 저장
with open('movie.csv', 'w', encoding='utf-8') as f:
    # 헤더와 딕셔너리 키 값을 맞춰줘야함.
    fieldnames = ['movieCd', 'movieNm', 'movieNmEn', 'movieNmOg', 'watchGradeNm', 'openDt', 'showTm', 'genreNm', 'peopleNm'] 
    csv_writer = csv.DictWriter(f, fieldnames=fieldnames)
    csv_writer.writeheader()
    for movie in movie_infos:
        csv_writer.writerow(movie)

In [16]:
# 결과 파일 확인
with open('movie.csv', 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        pprint.pprint(row)

OrderedDict([('movieCd', '20186202'),
             ('movieNm', '신과함께-인과 연'),
             ('movieNmEn', 'Along with the Gods: The Last 49 Days'),
             ('movieNmOg', ''),
             ('watchGradeNm', '12세이상관람가'),
             ('openDt', '20180801'),
             ('showTm', '141'),
             ('genreNm', '판타지'),
             ('peopleNm', '김용화')])
OrderedDict([('movieCd', '20181181'),
             ('movieNm', '미션 임파서블: 폴아웃'),
             ('movieNmEn', 'Mission:Impossible- Fallout'),
             ('movieNmOg', ''),
             ('watchGradeNm', '15세이상관람가'),
             ('openDt', '20180725'),
             ('showTm', '147'),
             ('genreNm', '액션'),
             ('peopleNm', '크리스토퍼 맥쿼리')])
OrderedDict([('movieCd', '20183361'),
             ('movieNm', '인크레더블 2'),
             ('movieNmEn', 'INCREDIBLES 2'),
             ('movieNmOg', ''),
             ('watchGradeNm', '전체관람가'),
             ('openDt', '20180718'),
             ('showTm', '125'),
             ('genreNm', 

# 영화인 정보

02에서 수집한 영화 감독정보를 활용해 상세 정보 수집.
향후 감독 정보로 활용될 예정.


* 요청 조건
    * 요청 변수 : key(필수), `영화인명 : peopleNm`으로 조회
    * peopleNm None 일수도
    
    
    
   
* 결과
    * 영화인별로 `영화인 코드 : peopleCd`, `영화인명 : peopleNm`, `분야 : repRoleNm`, `필모리스트 : filmoNames` 저장
    * **director.csv에 저장**
    (만약 검색 결과 없으면 저장하지 않기)
    

* [소스코드](./03.py)

In [2]:
# 영화인 상세 정보

import requests
import pprint
import csv

# 영화인 상세정보 딕셔너리가 들어갈 리스트
people_infos = [] 
key = '2c6010411226af12f598c9e149bfeca8'
api_url = f'http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleList.json?key={key}'

api_url은 key만 들어가게 한다.
반복문을 돌리면서 영화인명(peopleNm)으로 조회할 수 있게 한다.

In [5]:
with open('movie.csv', 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    # row는 한 영화의 상세정보를 담고 있음
    for row in reader:
        temp = {}
        # 만약 peopleCd == None 이면 다음 row로 넘어감
        if not row.get('peopleNm'):
            continue
        # url 요청해서 값 받아오기. 영화인명으로 검색한다.
        response = requests.get(api_url + '&peopleNm=' + row.get('peopleNm')).json()
        pprint.pprint(response)
        # 만약 검색 결과가 없으면 다음 row로 넘어감
        if not response.get('peopleListResult').get('peopleList'):
            continue
        for people in response.get('peopleListResult').get('peopleList'):
            # 영화 감독 이름이 일치하고, repRoleNm이 감독일 때 영화인 상세정보를 저장하고 반복문 종료
            if people.get('peopleNm') == row.get('peopleNm') and people.get('repRoleNm') == '감독':
                peopleCd = people.get('peopleCd')
                peopleNm = people.get('peopleNm')
                repRoleNm = people.get('repRoleNm')
                filmoNames = people.get('filmoNames')
                temp = {
                    'peopleCd': peopleCd,
                    'peopleNm': peopleNm,
                    'repRoleNm': repRoleNm,
                    'filmoNames': filmoNames
                }
                people_infos.append(temp)
                break

{'peopleListResult': {'peopleList': [{'filmoNames': '저격자 ',
                                      'peopleCd': '20115572',
                                      'peopleNm': '김용화',
                                      'peopleNmEn': '',
                                      'repRoleNm': '감독'},
                                     {'filmoNames': '백두산|신과함께-인과 연|신과함께-죄와 '
                                                    '벌|미스터 고|마이 웨이|국가대표 완결판-못 '
                                                    '다한 이야기|국가대표|미녀는 괴로워|오! '
                                                    '브라더스|자반고등어',
                                      'peopleCd': '10005151',
                                      'peopleNm': '김용화',
                                      'peopleNmEn': 'KIM Yong-hwa',
                                      'repRoleNm': '감독'},
                                     {'filmoNames': '분홍신',
                                      'peopleCd': '20165115',
                                      'p

{'peopleListResult': {'peopleList': [{'filmoNames': '맘마미아!2|나우이즈굿 |베스트 엑조틱 '
                                                    '메리골드 호텔|이매진 미 앤 유',
                                      'peopleCd': '10051071',
                                      'peopleNm': '올 파커',
                                      'peopleNmEn': 'Ol Parker',
                                      'repRoleNm': '감독'}],
                      'source': '영화진흥위원회',
                      'totCnt': 1}}
{'peopleListResult': {'peopleList': [{'filmoNames': '어느 가족|세 번째 살인|태풍이 '
                                                    '지나가고|바닷마을 다이어리|그렇게 아버지가 '
                                                    '된다|엔딩노트|진짜로 일어날지도 몰라 '
                                                    '기적|공기인형|걸어도 걸어도|하나|아무도 '
                                                    '모른다|디스턴스|원더풀 라이프|환상의 '
                                                    '빛|그러나... 복지를 버리는 시대로|또 '
                                                    '하나의 교육',
     

{'peopleListResult': {'peopleList': [{'filmoNames': '서치',
                                      'peopleCd': '10011005',
                                      'peopleNm': '아니쉬 차간티',
                                      'peopleNmEn': 'Aneesh Chaganty',
                                      'repRoleNm': '감독'}],
                      'source': '영화진흥위원회',
                      'totCnt': 1}}
{'peopleListResult': {'peopleList': [{'filmoNames': '상류사회|주홍글씨|서프라이즈|인터뷰|올슨|His '
                                                    'Concern|어머니의 여름|호모 비디오쿠스',
                                      'peopleCd': '10031397',
                                      'peopleNm': '변혁',
                                      'peopleNmEn': 'BYUN Daniel H.',
                                      'repRoleNm': '감독'}],
                      'source': '영화진흥위원회',
                      'totCnt': 1}}
{'peopleListResult': {'peopleList': [{'filmoNames': '로마|그래비티|칠드런 오브 맨|해리포터와 '
                                            

{'peopleListResult': {'peopleList': [{'filmoNames': '더 넌',
                                      'peopleCd': '10076618',
                                      'peopleNm': '코린 하디',
                                      'peopleNmEn': 'Corin Hardy',
                                      'repRoleNm': '감독'}],
                      'source': '영화진흥위원회',
                      'totCnt': 1}}
{'peopleListResult': {'peopleList': [{'filmoNames': '극장판 요괴워치 섀도사이드: 도깨비왕의 '
                                                    '부활|극장판 요괴워치: 하늘을 나는 고래와 '
                                                    '더블세계다냥!|극장판 요괴워치: 탄생의 '
                                                    '비밀이다냥!|스트레이트 재킷',
                                      'peopleCd': '20171642',
                                      'peopleNm': '우시로 신지',
                                      'peopleNmEn': 'Shinji Ushiro',
                                      'repRoleNm': '감독'}],
                      'source': '영화진흥위원회',
                  

{'peopleListResult': {'peopleList': [{'filmoNames': '곰돌이 푸 다시 만나 행복해|월드 워 '
                                                    'Z|머신건 프리처 감독판|머신건 프리처|007 '
                                                    '퀀텀 오브 솔러스|연을 쫓는 아이|스트레인저 '
                                                    '댄 픽션|스테이|네버랜드를 찾아서|몬스터 볼',
                                      'peopleCd': '10024440',
                                      'peopleNm': '마크 포스터',
                                      'peopleNmEn': 'Marc Forster',
                                      'repRoleNm': '감독'}],
                      'source': '영화진흥위원회',
                      'totCnt': 1}}
{'peopleListResult': {'peopleList': [{'filmoNames': '모차르트: 프라하의 서곡|모래요정과 '
                                                    '아이들|애니멀 팜',
                                      'peopleCd': '10068082',
                                      'peopleNm': '존 스티븐슨',
                                      'peopleNmEn': 'John Stephenson',
                      

...

{'peopleListResult': {'peopleList': [{'filmoNames': '위크엔드 인 파리|굿모닝 에브리원|사랑을 '
                                                    '견뎌내기|마더|체인징 레인스|노팅 힐|설득',
                                      'peopleCd': '10017730',
                                      'peopleNm': '로저 미첼',
                                      'peopleNmEn': 'Roger Michell',
                                      'repRoleNm': '감독'}],
                      'source': '영화진흥위원회',
                      'totCnt': 1}}
{'peopleListResult': {'peopleList': [{'filmoNames': '나의 특별한 형제|화장|강철대오:구국의 '
                                                    '철가방|방가? 방가!|여자, 정혜|달마야, '
                                                    '서울 가자|아이언 팜|축제|장미빛 인생',
                                      'peopleCd': '10054185',
                                      'peopleNm': '육상효',
                                      'peopleNmEn': 'YOOK Sang-hyo',
                                      'repRoleNm': '감독'}],
                      'source': '영화

{'peopleListResult': {'peopleList': [{'filmoNames': '교회오빠',
                                      'peopleCd': '20322109',
                                      'peopleNm': '이호경',
                                      'peopleNmEn': '',
                                      'repRoleNm': '감독'},
                                     {'filmoNames': '그녀들의 방|어떤 개인 날|지금, 이대로가 '
                                                    '좋아요',
                                      'peopleCd': '20151953',
                                      'peopleNm': '이호경',
                                      'peopleNmEn': '',
                                      'repRoleNm': '스크립터'},
                                     {'filmoNames': '내 여자의 남자친구',
                                      'peopleCd': '20158352',
                                      'peopleNm': '이호경',
                                      'peopleNmEn': '',
                                      'repRoleNm': '배우'},
                                   

{'peopleListResult': {'peopleList': [{'filmoNames': '로켓맨|잇 로컬|독수리 에디|선샤인 온 '
                                                    '리스|와일드 빌|스탠더|록 스탁 앤 2 스모킹 '
                                                    '배럴즈|죽어도 좋아',
                                      'peopleCd': '10012399',
                                      'peopleNm': '덱스터 플레처',
                                      'peopleNmEn': 'Dexter Fletcher',
                                      'repRoleNm': '배우'}],
                      'source': '영화진흥위원회',
                      'totCnt': 1}}
{'peopleListResult': {'peopleList': [{'filmoNames': '트랩: 디렉터스 컷|백야행|미성년자 관람불가',
                                      'peopleCd': '10028891',
                                      'peopleNm': '박신우',
                                      'peopleNmEn': 'PARK Sin-woo',
                                      'repRoleNm': '감독'},
                                     {'filmoNames': '원펀치|두 사람이다',
                                      'peopleCd': '20

{'peopleListResult': {'peopleList': [{'filmoNames': '사탄의 인형|폴라로이드',
                                      'peopleCd': '10008737',
                                      'peopleNm': '라스 클리브버그',
                                      'peopleNmEn': 'Lars Klevberg',
                                      'repRoleNm': '감독'}],
                      'source': '영화진흥위원회',
                      'totCnt': 1}}
{'peopleListResult': {'peopleList': [{'filmoNames': '애나벨 집으로',
                                      'peopleCd': '10018162',
                                      'peopleNm': '게리 도버먼',
                                      'peopleNmEn': 'Gary Dauberman',
                                      'repRoleNm': '감독'}],
                      'source': '영화진흥위원회',
                      'totCnt': 1}}
{'peopleListResult': {'peopleList': [{'filmoNames': '독전|신 전래동화|그것만이 내 '
                                                    '세상|강철비|남한산성|로마의 휴일|불한당: '
                                                    '나쁜 놈

{'peopleListResult': {'peopleList': [{'filmoNames': '극장판 엉덩이 탐정: 화려한 사건 수첩',
                                      'peopleCd': '10090941',
                                      'peopleNm': '히라야마 미호',
                                      'peopleNmEn': 'Miho Hirayama',
                                      'repRoleNm': '감독'}],
                      'source': '영화진흥위원회',
                      'totCnt': 1}}
{'peopleListResult': {'peopleList': [{'filmoNames': '미드소마|유전',
                                      'peopleCd': '10021926',
                                      'peopleNm': '아리 애스터',
                                      'peopleNmEn': 'Ari Aster',
                                      'repRoleNm': '감독'}],
                      'source': '영화진흥위원회',
                      'totCnt': 1}}


In [6]:
# director.csv에 저장하기
with open('director.csv', 'w', encoding='utf-8') as f:
    # 헤더와 딕셔너리 키 값을 맞춰줘야함.
    fieldnames = ['peopleCd', 'peopleNm', 'repRoleNm', 'filmoNames'] 
    csv_writer = csv.DictWriter(f, fieldnames=fieldnames)
    csv_writer.writeheader()
    for people_info in people_infos:
        csv_writer.writerow(people_info)

In [7]:
# 결과 파일 확인
with open('director.csv', 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        pprint.pprint(row)

OrderedDict([('peopleCd', '20115572'),
             ('peopleNm', '김용화'),
             ('repRoleNm', '감독'),
             ('filmoNames', '저격자 ')])
OrderedDict([('peopleCd', '10077515'),
             ('peopleNm', '크리스토퍼 맥쿼리'),
             ('repRoleNm', '감독'),
             ('filmoNames',
              '미션 임파서블: 폴아웃|미션 임파서블: 로그네이션|잭 리처|웨이 오브 더 건|유주얼 서스펙트')])
OrderedDict([('peopleCd', '10032042'),
             ('peopleNm', '브래드 버드'),
             ('repRoleNm', '감독'),
             ('filmoNames',
              '인크레더블 2|투모로우랜드|미션임파서블:고스트프로토콜|라따뚜이|인크레더블|심슨 가족|아이언 자이언트')])
OrderedDict([('peopleCd', '20306533'),
             ('peopleNm', '최신규'),
             ('repRoleNm', '감독'),
             ('filmoNames',
              '극장판 헬로카봇 : 달나라를 구해줘!|극장판 헬로카봇: 옴파로스 섬의 비밀|극장판 공룡메카드: 타이니소어의 '
              '섬|극장판 헬로카봇 : 백악기 시대|헬로 카봇')])
OrderedDict([('peopleCd', '10004265'),
             ('peopleNm', '김병갑'),
             ('repRoleNm', '감독'),
             ('filmoNames',
              '신비아파트: 금빛 도깨비와 비밀의 동굴|마